# Parameter Efficient Fine-Tuning with LoRa





This exercise uses code from the notebook authored by Achilles Moraites. The orignal notebook can be found  **[here](https://github.com/achimoraites/machine-learning-playground/blob/main/NLP/Text%20classification/Lightweight_RoBERTa_PEFT_LORA_FineTuning.ipynb)**. The Fine-tuning dataset is ag_news. We hope this notebook, with our added comments, serves as reference for you to understand the possiblities with LoRA.

######Copyright (c) 2021 Achilles Moraites



Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


## Install dependencies




In [1]:
#!pip install transformers datasets evaluate accelerate peft - Shortcut to use rathe rather than below.

# Install the required packages using pip
!pip install transformers  # Library for transformer-based models like BERT, RoBERTa
!pip install datasets  # Library for accessing and managing datasets for NLP tasks
!pip install evaluate  # Library for evaluating model performance on various tasks
!pip install accelerate  # Library for accelerating training and inference with PyTorch
!pip install peft  # Pretrained Error-Fix Transformer (PEFT) for improving language models



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 45.3 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


## Exercise #1: Dataset Preprocessing


In [2]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
from datasets import load_dataset


# Define model names


# Load dataset


# Initialize tokenizer using the base RoBERTa model


# Define a preprocessing function


# Apply preprocessing to the entire dataset in batches, removing the 'text' column



# Define model names
roberta_model_name = 'roberta-base-peft'  # PEFT stands for Pretrained Error-Fix Transformer
modified_base = 'roberta-base-modified'  # A modified version of the RoBERTa base model
base_model = 'roberta-base'  # The original RoBERTa base model

# Load dataset
dataset = load_dataset('ag_news')  # Load the AG News dataset

# Initialize tokenizer using the base RoBERTa model
tokenizer = RobertaTokenizer.from_pretrained(base_model)

# Define a preprocessing function
def preprocess(examples):
    tokenized = tokenizer(examples['text'], truncation=True, padding=True)   # Tokenize the text data, truncating and padding sequences
    return tokenized

# Apply preprocessing to the entire dataset in batches, removing the 'text' column
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])

# Split the tokenized dataset into train, evaluation, and test datasets
train_dataset = tokenized_dataset['train']  # Training dataset
eval_dataset = tokenized_dataset['test'].shard(num_shards=2, index=0)  # Evaluation dataset (1st shard)
test_dataset = tokenized_dataset['test'].shard(num_shards=2, index=1)  # Test dataset (2nd shard)


# Extract the number of classess and their names
num_labels = dataset['train'].features['label'].num_classes
class_names = dataset["train"].features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


# # Split the tokenized dataset into train, evaluation, and test datasets
# train_dataset = tokenized_dataset['train']  # Training dataset
# eval_dataset = tokenized_dataset['test'].shard(num_shards=2, index=0)  # Evaluation dataset (1st shard)
# test_dataset = tokenized_dataset['test'].shard(num_shards=2, index=1)  # Test dataset (2nd shard)


# # Extract the number of classess and their names
# num_labels = dataset['train'].features['label'].num_classes
# class_names = dataset["train"].features["label"].names
# print(f"number of labels: {num_labels}")
# print(f"the labels: {class_names}")

# # Create an id2label mapping
# # We will need this for our classifier.
# id2label = {i: label for i, label in enumerate(class_names)}

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


## Training

Let's Train two models, one using LORA and the other with full fine-tuning.

Note the LORA training times and the number of trained parameters!

In [3]:
# use the same Training args for all models

# Define training arguments for model training
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save the model and training results
    evaluation_strategy='steps',  # Evaluate on the validation set every 'steps' training steps
    learning_rate=5e-5,  # Initial learning rate for the optimizer
    num_train_epochs=1,  # Number of times to iterate over the entire training dataset
    per_device_train_batch_size=16,  # Batch size for each GPU during training
)


/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
# Define a function to create a Trainer object for model training
def get_trainer(model):
    # Create a Trainer object with the specified arguments
    return Trainer(
        model=model,  # The model to be trained
        args=training_args,  # Training arguments for configuring the training process
        train_dataset=train_dataset,  # Training dataset
        eval_dataset=eval_dataset,  # Evaluation dataset
        data_collator=data_collator,  # Data collator for batch processing
    )

In [5]:

# Create a Trainer object for full fine-tuning of the model
full_finetuning_trainer = get_trainer(
    AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label),
)
# Start training the model using the Trainer object
full_finetuning_trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


: 

## Exercise #2: PEFT Training

In [ ]:

# Creating a model for sequence classification based on a pretrained model
model = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label)

# Configuring Lora parameters for PEFT (Proxy Experience Finetuning)
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)

# Applying Lora parameters to the model to create the PEFT model
peft_model = get_peft_model(model, peft_config)

# Printing information about the PEFT model's trainable parameters
print('PEFT Model')
peft_model.print_trainable_parameters()

# Obtaining a trainer object for Lora finetuning
peft_lora_finetuning_trainer = get_trainer(peft_model)

# Initiating the training process for the PEFT model using Lora finetuning
peft_lora_finetuning_trainer.train()




NameError: name 'AutoModelForSequenceClassification' is not defined

In [ ]:
# Save
tokenizer.save_pretrained(modified_base)
peft_model.save_pretrained(peft_model_name)

## Performing Inference with a PEFT Model

It's time to have some fun putting our model to work!

In [ ]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer


# LOAD the Saved PEFT model

inference_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_name, id2label=id2label)
tokenizer = AutoTokenizer.from_pretrained(modified_base)

def classify(text):
    # Tokenize the input text, truncate if needed, pad sequences, and convert to PyTorch tensors
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    # Pass the tokenized input through the inference model to get predictions
    output = inference_model(**inputs)

    # Extract the predicted class index from the model output
    prediction = output.logits.argmax(dim=-1).item()

  print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
  # return id2label[prediction]


In [ ]:
classify( "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...")
classify( "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")

### Evaluate Models

To measure the improvement of the Training process we will need a baseline; let's compare the trained models with an untrained one.

Take a Look how the trained models perform

1.   The trained models against the untrained one
2.   The PEFT Model vs the regular fine-tuned one  



In [ ]:

# Import necessary modules
from torch.utils.data import DataLoader  # DataLoader for creating data iterators
import evaluate  # Import module for computing evaluation metrics
from tqdm import tqdm  # tqdm for progress tracking during evaluation

# Load the evaluation metric (e.g., accuracy) from the evaluate module
metric = evaluate.load('accuracy')

# Define a function to evaluate the model on a given dataset
def evaluate_model(inference_model, dataset):
    # Create a DataLoader for the evaluation dataset with batch size 8 and using data_collator for batch processing
    eval_dataloader = DataLoader(dataset.rename_column("label", "labels"), batch_size=8, collate_fn=data_collator)
    # Determine the device (GPU or CPU) for model inference
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the inference model to the appropriate device (GPU or CPU)
    inference_model.to(device)
    # Set the model in evaluation mode (no gradient computation)
    inference_model.eval()

    # Iterate over batches in the evaluation dataloader
    for step, batch in enumerate(tqdm(eval_dataloader)):
        # Move the batch to the device (GPU or CPU)
        batch.to(device)
        # Disable gradient calculation for inference
        with torch.no_grad():
            # Perform inference using the inference model on the batch
            outputs = inference_model(**batch)
        # Extract the predicted class indices from the model outputs
        predictions = outputs.logits.argmax(dim=-1)
        # Assign predictions and reference labels to variables for metric computation
        predictions, references = predictions, batch["labels"]
        # Add the batch of predictions and references to the evaluation metric
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    # Compute the evaluation metric (e.g., accuracy) based on added batches
    eval_metric = metric.compute()
    # Print the computed evaluation metric
    print(eval_metric)



In [ ]:
# Evaluate the non fine-tuned model
evaluate_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label), test_dataset)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 475/475 [00:13<00:00, 35.48it/s]

{'accuracy': 0.24868421052631579}


In [ ]:
# Evaluate the PEFT fine-tuned model
evaluate_model(inference_model, test_dataset)

100%|██████████| 475/475 [00:13<00:00, 34.20it/s]

{'accuracy': 0.9278947368421052}


In [ ]:
# Evaluate the Fully fine-tuned model
evaluate_model(full_finetuning_trainer.model, test_dataset)

100%|██████████| 475/475 [00:13<00:00, 35.56it/s]

{'accuracy': 0.9460526315789474}
